<a href="https://colab.research.google.com/github/krzysztofnowakuz/AGH/blob/main/Decyclization_%26_Quick_heuristic_recovery_policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd

data=pd.read_csv("mainForm.txt", sep=" ", header=None)
tabCol=[]
tabRow=[]

for col in data.columns:
    tabCol.append( "P" + str(col+1))
data.columns = tabCol

for row in data.index-2:
    tabRow.append("T" + str(row+1))
data.index=tabRow

incidenceMatrix = data.replace('x', -1)

initialMarking = incidenceMatrix.head(2)
initialMarking = initialMarking.to_numpy()
incidenceMatrix = incidenceMatrix[2:]

incidenceMatrix = incidenceMatrix.astype(int)
numEdges, numVertices = incidenceMatrix.shape

print("Incidence matrix")
print(incidenceMatrix)

Incidence matrix
    P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  P11
T1  -1   1   0   0   0   0   0   0  -1    0    0
T2   0  -1   1   0   0   0   0   0   1   -1    0
T3   0   0  -1   1   0   0   0   0   0    1   -1
T4   1   0   0  -1   0   0   0   0   0    0    1
T5   0   0   0   0   1   0   0  -1   0    0   -1
T6   0   0   0   0  -1   1   0   0   0   -1    1
T7   0   0   0   0   0  -1   1   0  -1    1    0
T8   0   0   0   0   0   0  -1   1   1    0    0


#Initial marking

In [24]:
print(initialMarking)

[['i' 'a' 'a' 'a' 'a' 'a' 'a' 'i' 'r' 'r' 'r']
 ['3' '0' '0' '0' '0' '0' '0' '3' '1' '1' '1']]


In [26]:
vertices = numVertices
vertices = list(range(1, vertices + 1))

edges = []

for row in incidenceMatrix.values:
    tails = [i+1 for i, x in enumerate(row) if x == -1]
    heads = [i+1 for i, x in enumerate(row) if x == 1]
    edges.extend([(tail, head) for tail in tails for head in heads])

print("Vertices =", vertices)
print("Edges =", edges)

Vertices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Edges = [(1, 2), (9, 2), (2, 3), (2, 9), (10, 3), (10, 9), (3, 4), (3, 10), (11, 4), (11, 10), (4, 1), (4, 11), (8, 5), (11, 5), (5, 6), (5, 11), (10, 6), (10, 11), (6, 7), (6, 10), (9, 7), (9, 10), (7, 8), (7, 9)]


In [27]:
head = [e[0] for e in edges]
tail = [e[1] for e in edges]

for e in edges:
    head[e[0]] += 1
    tail[e[1]] += 1

w = {}

for e in edges:
    w[e] = tail[e[1]] - head[e[1]] + head[e[0]] - tail[e[0]]

print(w)
print('Head', head)
print('Tail', tail)

{(1, 2): 8, (9, 2): 2, (2, 3): 6, (2, 9): -2, (10, 3): 10, (10, 9): 2, (3, 4): -14, (3, 10): -10, (11, 4): -14, (11, 10): -10, (4, 1): 0, (4, 11): 14, (8, 5): 6, (11, 5): -8, (5, 6): 2, (5, 11): 8, (10, 6): 4, (10, 11): 10, (6, 7): 6, (6, 10): -4, (9, 7): 8, (9, 10): -2, (7, 8): -14, (7, 9): -8}
Head [1, 10, 4, 4, 12, 12, 5, 5, 12, 14, 8, 7, 8, 11, 5, 5, 10, 10, 6, 6, 9, 9, 7, 7]
Tail [2, 3, 5, 11, 5, 11, 6, 12, 5, 13, 5, 14, 5, 5, 6, 11, 6, 11, 7, 10, 7, 10, 8, 9]


In [28]:
sorted_edges = sorted(w.items(), key=lambda x: x[1])
for edge, weight in sorted_edges:
    print(f'Edge: {edge}, Weight: {weight}')

Edge: (3, 4), Weight: -14
Edge: (11, 4), Weight: -14
Edge: (7, 8), Weight: -14
Edge: (3, 10), Weight: -10
Edge: (11, 10), Weight: -10
Edge: (11, 5), Weight: -8
Edge: (7, 9), Weight: -8
Edge: (6, 10), Weight: -4
Edge: (2, 9), Weight: -2
Edge: (9, 10), Weight: -2
Edge: (4, 1), Weight: 0
Edge: (9, 2), Weight: 2
Edge: (10, 9), Weight: 2
Edge: (5, 6), Weight: 2
Edge: (10, 6), Weight: 4
Edge: (2, 3), Weight: 6
Edge: (8, 5), Weight: 6
Edge: (6, 7), Weight: 6
Edge: (1, 2), Weight: 8
Edge: (5, 11), Weight: 8
Edge: (9, 7), Weight: 8
Edge: (10, 3), Weight: 10
Edge: (10, 11), Weight: 10
Edge: (4, 11), Weight: 14


In [6]:
def is_path_exists(graph, start, end):
    visited = set()
    queue = [start]

    while queue:
        vertex = queue.pop(0)
        if vertex == end:
            return True
        if vertex not in visited:
            visited.add(vertex)
            queue.extend(graph.get(vertex, []))
    return False

G_prime = {}
E_prime = []

for edge, weight in sorted_edges:
    head, tail = edge
    if not is_path_exists(G_prime, head, tail):
        E_prime.append(edge)
        if head in G_prime:
            G_prime[head].append(tail)
        else:
            G_prime[head] = [tail]
print("E'",E_prime)
print("G'",G_prime)

E' [(3, 4), (11, 4), (7, 8), (3, 10), (11, 10), (11, 5), (7, 9), (6, 10), (2, 9), (9, 10), (4, 1), (9, 2), (10, 9), (5, 6), (10, 6), (2, 3), (8, 5), (6, 7), (1, 2), (5, 11)]
G' {3: [4, 10], 11: [4, 10, 5], 7: [8, 9], 6: [10, 7], 2: [9, 3], 9: [10, 2], 4: [1], 10: [9, 6], 5: [6, 11], 8: [5], 1: [2]}
